<a href="https://colab.research.google.com/github/DipankarJDutta/DipankarJDutta-Biomarker-discovery-from-B-T-cells-in-FASD-GDM-with-LSTM/blob/master/LSTM_B_T_Median_Z-score_base-model_hyperparameter%20optimized.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Print versions of Backends
from keras import backend as K
from distutils.version import LooseVersion as LV
from keras import __version__
if K.backend() == "tensorflow":
    import tensorflow as tf
    device_name = tf.test.gpu_device_name()
    if device_name == '':
        device_name = "None"
    print('Using TensorFlow version:', tf.__version__, ', GPU:', device_name)
print('Using Keras version:', __version__, 'backend:', K.backend())
assert(LV(__version__) >= LV("2.0.0"))

Using TensorFlow backend.


Using TensorFlow version: 1.15.0 , GPU: None
Using Keras version: 2.2.5 backend: tensorflow


In [0]:
# Loading essentials
import numpy as np
from numpy import loadtxt
from numpy import reshape
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

In [0]:
# Fixing random seed to 007 for reproducibility
from numpy.random import seed
seed(7)
from tensorflow import set_random_seed
set_random_seed(7)

In [4]:
# Load data
dataset = loadtxt('B-T-median_z-score.csv', delimiter = ',')
print (dataset)
print(np.size(dataset))

[[-1.15896943  0.24160534  0.44608631 ... -0.97378438  0.2817533
   1.        ]
 [-0.16010128  0.05440921  0.1136581  ... -1.98427671 -2.25947407
   1.        ]
 [ 0.76935912  1.34486762  0.18530212 ... -1.98427671 -2.25947407
   0.        ]
 ...
 [ 0.76935912  0.64387787 -1.3994634  ... -1.98427671 -2.25947407
   0.        ]
 [ 0.76935912  0.81115951 -0.77186186 ... -1.98427671 -2.25947407
   0.        ]
 [ 0.76935912  0.70362131 -1.61439543 ...  0.41264309  0.43123726
   1.        ]]
1680


In [0]:
# Reshape 2D dataset into a 3D dataset with columns as features with one time-step
x = dataset.reshape(56, 1, 30)

In [6]:
# Confirm data shape of 56 samples (rows), 1 time-step per feature (columns as features with one time-step), 29 features/z-scored inclevel gene values (16 from B-cells & 13 from T-cells, in that order) & the last 1 column as binary output of good (1) and bad (0) learners as differentiated by the population median.
print(x.shape)
print(x.size)

(56, 1, 30)
1680


In [7]:
#print x
print(x)

[[[-1.15896943  0.24160534  0.44608631 ... -0.97378438  0.2817533
    1.        ]]

 [[-0.16010128  0.05440921  0.1136581  ... -1.98427671 -2.25947407
    1.        ]]

 [[ 0.76935912  1.34486762  0.18530212 ... -1.98427671 -2.25947407
    0.        ]]

 ...

 [[ 0.76935912  0.64387787 -1.3994634  ... -1.98427671 -2.25947407
    0.        ]]

 [[ 0.76935912  0.81115951 -0.77186186 ... -1.98427671 -2.25947407
    0.        ]]

 [[ 0.76935912  0.70362131 -1.61439543 ...  0.41264309  0.43123726
    1.        ]]]


In [8]:
#Specify binary output of good (1) and bad (0) learners, y, in x & print y. Learner type differentiated by population median.
y = x [:, :, -1]
print(y)

[[1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]]


In [9]:
# Define LSTM One-to-One Model with 3 LSTM cells (determined via hyperparameter optimization). Sigmoid activation on the output layer for binary classification.
model = Sequential()
model.add(LSTM(3, input_shape=(1,30)))
model.add(Dense(1, activation='sigmoid'))

In [10]:
#Compile the model 
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [11]:
#Fit the model with a 80-20 split of dataset & shuffling sample order within an epoch. Batch size updated to 8 to be an even factor of sample size of 56.
model.fit(x, y, validation_split = 0.2, batch_size = 8, epochs = 1000, shuffle = True)




Train on 44 samples, validate on 12 samples
Epoch 1/1000





44/44 [==============================] - 1s 24ms/step - loss: 0.7022 - acc: 0.4773 - val_loss: 0.7315 - val_acc: 0.1667
Epoch 2/1000
44/44 [==============================] - 0s 630us/step - loss: 0.6997 - acc: 0.5000 - val_loss: 0.7304 - val_acc: 0.1667
Epoch 3/1000
44/44 [==============================] - 0s 443us/step - loss: 0.6974 - acc: 0.5000 - val_loss: 0.7295 - val_acc: 0.1667
Epoch 4/1000
44/44 [==============================] - 0s 446us/step - loss: 0.6952 - acc: 0.5000 - val_loss: 0.7287 - val_acc: 0.1667
Epoch 5/1000
44/44 [==============================] - 0s 508us/step - loss: 0.6929 - acc: 0.5227 - val_loss: 0.7272 - val_acc: 0.1667
Epoch 6/1000
44/44 [==============================] - 0s 547us/step - loss: 0.6908 - acc: 0.5682 - val_loss: 0.7256 - val_acc: 0.2500
Epoch 7/1000
44/44 [==============================] - 0s 518us/step - loss: 0.6888 - acc: 0.5909 - val_loss: 0.7241 - val_acc: 0.2500
Epoch 8/100

In [12]:
#Model Summary
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 3)                 408       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 4         
Total params: 412
Trainable params: 412
Non-trainable params: 0
_________________________________________________________________
None


In [13]:
#Installing h5py Python library to save trained models
!pip install h5py

In [0]:
#Save final keras LSTM Model
model.save('B-T-median_LSTM.h5')